# ASSIGNMENT 3 CH4 MOLECULE
# MMM 2025 - 12.3.2025
## Daniele Passerone




## Preparation

Run a Geo Opt and STM (ORBITALS) calculations for CH4.


For the analysis of the simulations, you will need to keep track of the pk of the simulation, pk of the trajectory and so on, as explained in the notebook 2/3.


In [ ]:
#
# some important Imports...
#
import numpy as np
from ase import Atoms
from ase.io import read
from ase.visualize import view
import matplotlib.pyplot as plt
import nglview as nv
from show_orbitals import all_orbitals

#
# and definitions of visualization functions (see last exercises)
#
def view_structure(structure,myvec=[]):
    t = nv.ASEStructure(structure)
    w = nv.NGLWidget(t, gui=True)
    w.add_unitcell()
    w.add_ball_and_stick()
    w.add_representation('label',label_type='atomindex',color='black')
    w.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
    return w

def view_trajectory(trajectory,myvec=[]):
    t2 = nv.ASETrajectory(trajectory)
    w2 = nv.NGLWidget(t2, gui=True)
    #w2.add_unitcell()
    w2.add_ball_and_stick()
    w2.add_representation('spacefill',selection=myvec,color="blue",radius=0.5)
    return w2

## 1. Visualizing the optimization trajectory 
As shown in the **Notebook 2** you need the pk of the optimization trajectory, that you will insert in the next cell as value of **trajpk**.

In [ ]:

%load_ext aiida
%aiida

molname = 'ch4'
from aiida.orm import load_node

my_geo_opt_pk = XX # THE PK OF THE GEO OPT
my_pk = XXX # THE ONE OF THE ORBITALS
my_numberocc = xxxx # THE NUMBER OF OCCUPIED ORBITALS IN THE SYSTEM, (N valence electrons/2)

workchain = load_node(my_geo_opt_pk)

# Access the TrajectoryData node
trajectory_data = workchain.outputs.output_trajectory  
trajectory_pk = trajectory_data.pk

print(f"The PK of the TrajectoryData is: {trajectory_pk}")


traj = load_node(trajectory_pk)
symbols = traj.symbols
trajase=[traj.get_step_structure(step-1).get_ase() for step in traj.get_stepids()]
for a in trajase:
    a.set_pbc([False,False,False])

In [ ]:
view_trajectory(trajase)

#
# The optimization trajectory is shown.
#

## 2. Visualizing the orbitals
Now we compute the orbitals. To this end we have already finished (see **Notebook 2**) the simulation of the orbitals, and hit the "Cube creation kit" button. Keep the **pk** of the SPM calculation ready.

In the Exercise_5 directory, there is a script "run_cube_from_wfn_acetylene.sh" that is able to take some files from the cp2k SPM simulation (wavefunction) and transform it into the orbital cube files. The important things in this file is the number of occupied and unoccupied orbitals, that should correspond to the ones that you have indicated when launching the SPM AiiDAlab workchain. In the case of acetylene, 10 electrons, 5 occupied orbitals.

In [ ]:
namefile = 'run_cube_from_wfn_'+molname+'.sh'
!cat {namefile}

## Generating the cubefiles, using the "cube-kit" set of files and the above script. 
Now we are ready to generate the orbital cube files. We replace my_pk below **with the pk of the SPM workchain**, and also the molecule name has to correspond.

In [ ]:
#
# Creating the cube file of the orbitals
# 

molecule = molname
!rm -Rf ./cube-kit-pk{my_pk}*
!cp /home/jovyan/apps/surfaces/tmp/cube-kit-pk{my_pk}.zip .
!unzip cube-kit-pk{my_pk}.zip
!cp run_cube_from_wfn_{molecule}.sh ./cube-kit-pk{my_pk}
!cd ./cube-kit-pk{my_pk} ; bash run_cube_from_wfn_{molecule}.sh 
!rm -Rf {molecule}_cubes
!mv ./cube-kit-pk{my_pk}/cubes {molecule}_cubes

print ("*****************************\n\nTHE GENERATED FILES ARE:\n")
!ls ./{molecule}_cubes
print ("\n*****************************\n")

## Visualizing HOMO and LUMO separately


We note that the name of the HOMO file above, we copy it into "file" and we read the cube file and its energy (change the name of the file accordingly)

In [ ]:
file = molecule + '_cubes/S0_6_HOMO-1.cube'
atoms = read(file)
a=!head -2 {file} | tail -1
b = str(a)
ene=(b[4:10])
view_homo=nv.NGLWidget()
caption_homo = "E="+ene+" eV\n"+"HOMO"
view_homo.add_component(nv.ASEStructure(atoms))
c_2 = view_homo.add_component(file)
c_2.clear()
c_2.add_surface(color='blue', isolevelType="value", isolevel=-0.01, opacity=0.05)
c_3 = view_homo.add_component(file)
c_3.clear()
c_3.add_surface(color='red', isolevelType="value", isolevel=0.01, opacity=0.05)


#
# And finally the visualization itself
#

print (caption_homo)
view_homo

We do the same for the LUMO, we simply need to change the name of the file.

In [ ]:
file = molecule + '_cubes/S0_8_LUMO.cube'
atoms = read(file)
a=!head -2 {file} | tail -1
b = str(a)
ene=(b[4:10])
view_lumo=nv.NGLWidget()
caption_lumo = "E="+ene+" eV\n"+'LUMO'
view_lumo.add_component(nv.ASEStructure(atoms))
c_2 = view_lumo.add_component(file)
c_2.clear()
c_2.add_surface(color='blue', isolevelType="value", isolevel=-0.01, opacity=0.05)
c_3 = view_lumo.add_component(file)
c_3.clear()
c_3.add_surface(color='red', isolevelType="value", isolevel=0.01, opacity=0.05)


#
# And finally the visualization itself
#
print (caption_lumo)
view_lumo

We now create a combined view that visualizes orbital and energy:

In [ ]:
import ipywidgets as widgets
widg_caption_homo = widgets.HTML(caption_homo)
combined_w_homo=widgets.HBox([view_homo,widg_caption_homo])

widg_caption_lumo = widgets.HTML(caption_lumo)
combined_w_lumo=widgets.HBox([view_lumo,widg_caption_lumo])


In [ ]:
combined_w_homo

In [ ]:
combined_w_lumo

## Visualizing all orbitals together

We will use a loop and arrays to caption all orbitals and plot a matrix of representations: see the file **show_orbitals.py** 

## Including the MATRIX VISUALIZATION OF ALL ORBITALS into a function

The function allows to choose the isosurface, and the first orbital to visualize in the array, as well as the last one. Note that for larger molecules (benzene) you better choose a few orbitals at the time, not to "kill" your jupyter.



In [ ]:
import show_orbitals

In [ ]:
#
# In this way, the molecule can be visualized with a call to the function followed by a call of the molecule itself
# See, in the Exercise directory, the file show_orbitals.py. Don't forget the total number of occupied orbitals (for naming)
#
!pwd
import importlib
importlib.reload(show_orbitals)
ch4 = show_orbitals.all_orbitals(molecule,pk=my_pk,nhomo=xxxx,nlumo=xxxx,ntotocc=my_numberocc,nfirstview=0,nlastview=9,isosurf=0.01);


In [ ]:
# Now, magically, the name of the molecule will visualize the orbitals


ch4

# Assignments

1. For each molecule, draw a molecular orbital table filling the orbitals up to the correct level.
2. Discuss the difference between the H2 and O2 molecule.
3. Discuss the differences between the CH4 and CH2  and C6H6  molecules (hybridisation?)
4. Follow the discussion that you find in the link, and compare with your result. Discuss the differences you find between Hexatriene and Benzene
5. Which molecule has the largest Band Gap?
6. Apply a deformation to benzene and optimize again. Show the trajectory of the optimization. 